### Article Selection
API calls are made making use of NY Times Developer platform. This will extract metadata and article snippets from NY Times archives based on the search conditions used. API will primarily use the news_desk attribute to import specific number of articles from each section. The sectional topics to be used are listed out in the developer website with over 100 topics available. Peripheral topics based on city/regional news, obituaries, job advertisments, classifieds, booming, crosswords etc. are excluded to work with only general topics.

In [1]:
import pandas as pd
import numpy as np

news_desk = pd.read_csv("data/news_desk.csv")

news_desk.head()

,Section
0,Adventure Sports
1,Arts & Leisure
2,Arts
3,Automobiles
4,Blogs


### API Calls
API calling will build a basic framework for extracting articles in bulk from the NY Times website. Due to the rate limiting restriction put up by NY Times, a decorater is used to make dynamic function calls so that the API requests do not exceed 1 call/second. There is also a day-wise limit of 1000 calls. An additional feature is added which will parallelize this process making use of multiple CPU cores and this will speed things up a bit. Brief description of Lock() function used for rate limiting can be found here.

In [9]:
import time, threading

def rate_limited(max_per_second):
  '''Decorator that make functions not to be called faster than 1 call/second'''
  lock = threading.Lock()
  minInterval = 1.0 / float(max_per_second)
  def decorate(func):
    lastTimeCalled = [0.0]
    def rateLimitedFunction(args,*kargs):
      lock.acquire()
      elapsed = time.clock() - lastTimeCalled[0]
      leftToWait = minInterval - elapsed
      if leftToWait>0:
        time.sleep(leftToWait)
      lock.release()
      ret = func(args,*kargs)
      lastTimeCalled[0] = time.clock()
      return ret
    return rateLimitedFunction
  return decorate


from threading import Thread
import requests

@rate_limited(0.9)
def process_id(id):
    try:
        r = requests.get(url % id)
        json_data = r.json()
        print('Appended '+str(page_index.index(id))+ ' out of '+ str(len(page_index)))
        return json_data
    except:
        json_data = ''
        print('Skipping...')
        return json_data

def process_range(id_range, store=None):
    if store is None:
        store = {}
    for id in id_range:
        store[id] = process_id(id)
    return store


def threaded_process_range(nthreads, id_range):
    store = {}
    threads = []

    for i in range(nthreads):
        ids = id_range[i::nthreads]
        t = Thread(target=process_range, args=(ids,store))
        threads.append(t)

    [t.start() for t in threads]
    [t.join() for t in threads]
    return store

news_desk = list(news_desk['Section'])
base_url = 'https://api.nytimes.com/svc/search/v2/articlesearch.json?api-key=a141a689509b459ba12e2b93b83883fd'

article_raw = []
for nd in news_desk[65:67]:
    param_url = '&fq=news_desk:'+str(nd)+'&sort=newest&page=%s'
    url = base_url + param_url
    print(str(nd)+":")
    page_index = list(range(7))
    try:
        articles_1 = threaded_process_range(2, page_index)
        articles_2 = [articles_1[k]['response']['docs'] for k in page_index if (type(articles_1[k]) is dict) and ('response' in articles_1[k])]
        articles_3 = [item for sublist in articles_2 for item in sublist]
        articles_4 = [{key:item[key] for key in ['web_url','pub_date']} for item in articles_3]
        articles_5 = pd.DataFrame(articles_4)
        articles_5['news_desk'] = str(nd)
        article_raw.append(articles_5)
    except:
        print('Skipping...')

url_data = pd.concat(article_raw).reset_index(drop = True)
#url_data.to_csv('data/url_data.csv', index = False)


Appended 1 out of 2
Appended 0 out of 2


In [119]:
from ipywidgets import widgets, HBox, VBox, Layout, Button, Label
from IPython.display import display
import requests

def onclick(sender):
    if(text.value == ''):
        print('Input cannot be blank')
        return
    
    base_url = 'https://api.nytimes.com/svc/search/v2/articlesearch.json?api-key=a141a689509b459ba12e2b93b83883fd'
    param_url = '&q='+str(text.value)+'&page=0'
    url = base_url + param_url
    
    r = requests.get(url)
    json_data = r.json()
    article_meta_data = json_data['response']['docs']
    
    headlines = []
    urls = []
    snippets = []
    for artc in article_meta_data:
        url = artc['web_url']
        headline = artc['headline']['main']
        snippet = artc['snippet']
        headlines.append(headline)
        urls.append(url)
        snippets.append(snippet)
    
    
    items_hl = [Button(description=w, border = 'solid', layout = Layout(width='80%', height='40px')) for w in headlines]
    for hl in items_hl:
        hl.style.button_color = 'SkyBlue'
        hl.style.font_weight = 'bold'
        
    items_sn = [widgets.Textarea(w, disabled = True, layout = Layout(width = '99.5%', height = '50px')) for w in snippets]

    items_rate = [widgets.IntSlider(
    value=1,
    min=1,
    max=5,
    step=1,
    description='More related articles?',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
    ) for i in range(len(article_meta_data))]
    
    items = [VBox([HBox([items_hl[i], items_rate[i]]), items_sn[i]]) for i in range(len(article_meta_data))]
    box = VBox(items, layout = Layout(border = 'solid'))
    display(box)
    


text_query = widgets.Text(placeholder='Type a query for NY Times article listing')
text_user = widgets.Text(placeholder='Enter your name', value = 'saket')
button_query = widgets.Button(description = 'Search articles')
query_input = HBox([text_query, button_query])
button_query.on_click(onclick)    

display(query_input) 
display(text_user)